In [3]:
!pip install -qq python-dotenv

In [4]:
# -*- coding: utf-8 -*-
import sys
import json
import os
import time
import requests
from dotenv import load_dotenv

# Carga de variables de entorno
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_ASSISTANT_ID = os.getenv("OPENAI_ASSISTANT_ID")


def run_assistant_http(assistant_id, user_input):
    """
    Ejecuta una conversación usando el API v2 de Assistants:
      1) Crea un thread global (POST /v1/threads)
      2) Añade mensaje del usuario (POST /v1/threads/{thread_id}/messages)
      3) Lanza el run con el assistant_id en el body (POST /v1/threads/{thread_id}/runs)
      4) Polling hasta estado 'completed'
      5) Recupera mensajes y devuelve el texto del primer bloque 'text'
    """
    headers = {
        "Content-Type": "application/json; charset=utf-8",
        "Authorization": f"Bearer {OPENAI_API_KEY}",
        "OpenAI-Beta": "assistants=v2"
    }

    # 1) Crear thread global
    t_resp = requests.post(
        "https://api.openai.com/v1/threads",
        headers=headers,
        json={},
        timeout=75
    )
    if t_resp.status_code not in (200, 201):
        return f"Error al crear thread: {t_resp.status_code} - {t_resp.text}"
    t_id = t_resp.json().get("id")

    # 2) Añadir mensaje del usuario (payload forzado en UTF-8)
    payload = json.dumps(
        {"role": "user", "content": user_input},
        ensure_ascii=False
    ).encode("utf-8")
    m_resp = requests.post(
        f"https://api.openai.com/v1/threads/{t_id}/messages",
        headers=headers,
        data=payload,
        timeout=75
    )
    if m_resp.status_code not in (200, 201):
        return f"Error al crear mensaje: {m_resp.status_code} - {m_resp.text}"

    # 3) Lanzar el run especificando assistant_id
    r_resp = requests.post(
        f"https://api.openai.com/v1/threads/{t_id}/runs",
        headers=headers,
        json={"assistant_id": assistant_id},
        timeout=75
    )
    if r_resp.status_code not in (200, 201):
        return f"Error al lanzar run: {r_resp.status_code} - {r_resp.text}"
    r_id = r_resp.json().get("id")

    # 4) Polling hasta que el run complete
    while True:
        time.sleep(1)
        status_resp = requests.get(
            f"https://api.openai.com/v1/threads/{t_id}/runs/{r_id}",
            headers=headers,
            timeout=75
        )
        if status_resp.status_code != 200:
            return f"Error al obtener estado: {status_resp.status_code} - {status_resp.text}"
        status = status_resp.json().get("status")
        if status == "completed":
            break
        if status == "failed":
            return "Ejecución fallida: status 'failed'."

    # 5) Recuperar mensajes del thread
    msgs_resp = requests.get(
        f"https://api.openai.com/v1/threads/{t_id}/messages",
        headers=headers,
        timeout=75
    )
    if msgs_resp.status_code != 200:
        return f"Error al recuperar mensajes: {msgs_resp.status_code} - {msgs_resp.text}"
    mensajes = msgs_resp.json().get("data", [])
    for msg in mensajes:
        if msg.get("role") == "assistant":
            bloques = msg.get("content", [])
            if bloques and bloques[0].get("type") == "text":
                return bloques[0]["text"].get("value", "")
    return "Sin respuesta del asistente."


def construir_prompt(operacion, p1, p2=None, valoracion=None, tema=None):
    if operacion == "UNION":
        return (
            f"Construye una nueva propuesta que incluye los elementos esenciales "
            f"de estas dos propuestas, eliminando las posibles redundancias, "
            f"de acuerdo al tema '{tema}':\n"
            f"Propuesta A: {p1}\n"
            f"Propuesta B: {p2} "
            "(No incluyas cabeceras, ni bullets, ni referencias a las propuestas de origen "
            "y mantén en lo posible la redacción y estilo de las propuestas originales, "
            "no inventando nuevos términos, ni sinónimos, ni giros, ni apreciaciones que "
            "no estén en las propuestas originales)"
        )
    elif operacion == "INTERSECCION":
        return (
            f"Construye una nueva propuesta que incluya solo los elementos que consideres "
            f"que son comunes a estas dos propuestas, eliminando la redundancia de lo común "
            f"y las disimilitudes, de acuerdo al tema '{tema}':\n"
            f"Propuesta A: {p1}\n"
            f"Propuesta B: {p2} "
            "(No incluyas cabeceras, ni bullets, ni referencias a las propuestas de origen "
            "y mantén en lo posible la redacción y estilo de las propuestas originales, "
            "no inventando nuevos términos, ni sinónimos, ni giros, ni apreciaciones que "
            "no estén en las propuestas originales)"
        )
    elif operacion == "CONSENSO":
        return (
            f"Redacta una nueva propuesta que consideres podría ser una propuesta de consenso "
            f"entre estas dos propuestas, de acuerdo al tema '{tema}':\n"
            f"Propuesta A: {p1}\n"
            f"Propuesta B: {p2} "
            "(No incluyas cabeceras, ni bullets, ni referencias a la propuesta de origen "
            "y mantén en lo posible la redacción y estilo de las propuestas originales, "
            "no inventando nuevos términos ni giros ni apreciaciones que no estén en las "
            "propuestas originales). Si ves que ambas propuestas están muy alejadas antecede "
            "la propuesta resultante con la frase 'El consenso es difícil con posturas tan alejadas, pero '"
        )
    elif operacion == "CERCANIA":
        return (
            f"¿Qué grado de similitud hay entre estas dos propuestas sobre '{tema}'? "
            "Proporciona solo un número entre 0 (nada similares o incluso opuestas) y 1 "
            "(idénticas o muy parecidas) y con dos decimales:\n"
            f"Propuesta A: {p1}\n"
            f"Propuesta B: {p2}"
        )
    elif operacion == "SIMPLIFICAR":
        return (
            f"Simplifica, resume y acorta la siguiente propuesta manteniendo su significado esencial:\n"
            f"{p1} "
            "(No incluyas cabeceras, ni bullets, ni referencias a la propuesta de origen "
            "y mantén en lo posible la redacción y estilo de la propuesta original, "
            "no inventando nuevos términos, ni sinónimos, ni giros, ni apreciaciones que "
            "no estén en las proposiciones originales)"
        )
    elif operacion == "GENERALIZAR":
        return (
            f"Reescribe la siguiente propuesta en términos más amplios y generales, "
            f"un poco más laxos que pudieran hacer que la apoyasen más participantes:\n"
            f"{p1} "
            "(No incluyas cabeceras, ni bullets, ni referencias a la propuesta de origen "
            "y mantén en lo posible la redacción y estilo de la propuesta original)"
        )
    elif operacion == "ENMIENDA+":
        return (
            f"Mejora la siguiente propuesta teniendo en cuenta esta valoración propositiva. "
            f"Intenta incluir algún matiz de acuerdo a la valoración aportada pero siempre "
            "que no desvirtúe la propuesta original:\n"
            f"Propuesta: {p1}\n"
            f"Valoración: {valoracion} "
            "(No incluyas cabeceras, ni bullets, ni referencias a la propuesta de origen "
            "y mantén en lo posible la redacción y estilo de la propuesta original)"
        )
    elif operacion == "ENMIENDA-":
        return (
            f"Corrige o atenúa la siguiente propuesta teniendo en cuenta esta crítica negativa. "
            f"Intenta incluir algún matiz de acuerdo a la valoración que atempere la razón de "
            "la crítica pero manteniendo la esencia de la propuesta original:\n"
            f"Propuesta: {p1}\n"
            f"Crítica: {valoracion} "
            "(No incluyas cabeceras, ni bullets, ni referencias a la propuesta de origen "
            "y mantén en lo posible la redacción y estilo de la propuesta original)"
        )
    elif operacion == "COHERENCIA":
        return (
            f"Evalúa la validez ortográfica y sintáctica de la siguiente propuesta y su coherencia con el tema '{tema}':\n"
            f"Propuesta: {p1}\n"
            "(En caso que la propuesta tenga más de cinco faltas de ortografía o es sintácticamente errónea retorna 'La propuesta no es válida ortográfica o sintácticamente' e indica a continuación cuáles son los fallos.) "
            "Además, si la propuesta manifiestamente no tiene nada que ver con el tema retorna 'La propuesta no es coherente con el tema o pregunta del debate'."
        )
    elif operacion == "TRIANGULO":
        return (
            "Crea una tercera propuesta de aproximadamente la misma longitud que las dos propuestas y que sea, en lo posible, bastante disimil con las dos propuestas de origen, como formando un triángulo de posturas con ambas, quizá considerando un aspecto que las otras dos no han considerado.\n"
            f"Tema: '{tema}'\n"
            f"Propuesta A: {p1}\n"
            f"Propuesta B: {p2}"
        )
    else:
        return "Operación no reconocida."


def resolver_operacion(operacion, p1, p2=None, valoracion=None, tema=None,
                       model="gpt-4", temperature=0.5, modo="prompting"):
    inicio = time.time()
    if modo == "asistente":
        if not OPENAI_ASSISTANT_ID:
            return {"resultado": "", "consola": "OPENAI_ASSISTANT_ID no definido"}
        user_input = json.dumps({
            "operacion": operacion,
            "p1": p1,
            "p2": p2,
            "valoracion": valoracion,
            "tema": tema
        }, ensure_ascii=False)
        contenido = run_assistant_http(OPENAI_ASSISTANT_ID, user_input)
        duracion = time.time() - inicio
        return {"resultado": contenido.strip(), "consola": f"Tiempo de ejecución: {duracion:.2f}s"}
    else:
        prompt = construir_prompt(operacion, p1, p2, valoracion, tema)
        try:
            chat_payload = json.dumps({
                "model": model,
                "messages": [{"role": "user", "content": prompt}],
                "temperature": temperature
            }, ensure_ascii=False).encode("utf-8")
            resp = requests.post(
                "https://api.openai.com/v1/chat/completions",
                headers={
                    "Content-Type": "application/json; charset=utf-8",
                    "Authorization": f"Bearer {OPENAI_API_KEY}"
                },
                data=chat_payload,
                timeout=75
            )
            duracion = time.time() - inicio
            if resp.status_code == 200:
                data = resp.json()
                texto = data["choices"][0]["message"]["content"].strip()
                return {"resultado": texto, "consola": f"Tiempo de ejecución: {duracion:.2f}s"}
            else:
                return {"resultado": "", "consola": f"Error {resp.status_code}: {resp.text}"}
        except Exception as e:
            return {"resultado": "", "consola": f"Error petición: {str(e)}"}




In [5]:
p1="La defensa tradicional de las corridas de toros parte de la convicción de que este espectáculo encarna una herencia cultural única, tejida a lo largo de siglos en España y países latinoamericanos. Para sus defensores, la tauromaquia es una expresión artística donde el torero y el toro participan en un ritual cargado de simbolismo, vestuario y música que transmite emociones profundas. Además, las plazas se convierten en epicentros de convivencia vecinal, fortaleciendo el sentido de comunidad durante ferias y fiestas patronales. En términos económicos, la industria taurina sustenta miles de empleos directos e indirectos —desde ganaderos y toreros hasta hosteleros y artesanos del traje de luces— y atrae turismo especializado que dinamiza zonas rurales a menudo olvidadas. Así, quienes abogan por mantener las corridas sostienen que, lejos de ser un mero espectáculo, preservan tradiciones y revitalizan economías locales sin parangón."
operacion="COHERENCIA"
tema="¿Deberían prohibirse las corridas de toros?"
model="gpt-4.5-preview"
temperature=1
modo="prompting"
resolver_operacion(operacion,p1,None,None,tema,model,temperature,modo)


{'resultado': 'La propuesta es válida ortográfica y sintácticamente, pues no contiene fallas ortográficas ni sintácticas evidentes.\n\nAdemás, la propuesta sí es coherente con el tema planteado "¿Deberían prohibirse las corridas de toros?", porque claramente presenta argumentos desde la perspectiva de quienes defienden su preservación, mencionando elementos culturales, económicos y sociales relacionados.\n\nConclusión:  \nLa propuesta es válida ortográfica y sintácticamente, y es coherente con el tema del debate.',
 'consola': 'Tiempo de ejecución: 4.67s'}